In [1]:
from glob import glob
from tqdm import tqdm
import json

In [2]:
import json

with open('news-30k.json') as fopen:
    news = json.load(fopen)

In [3]:
from glob import glob

files = sorted(glob('news/*.json'))
files = [f for f in files if 'populate-news' not in f]
len(files)

974

In [5]:
selected = []
already = set()

malaysian_news = {
    'kosmo',
    'hmetro',
    'malaymail',
    'projekmm',
    'bharian',
    'utusan',
    'astroawani',
    'themalaysianinsight',
    'malaysiakini',
    'bernama'
}

def reject(data):
    if data['news'] in malaysian_news:
        return False
    if any([n in data['top-image'] for n in malaysian_news]):
        return False
    if any([n in data['url'] for n in malaysian_news]):
        return False
    if 'com.my' in data['top-image']:
        return False
    if data['language'] == 'malay':
        return False
    if 'Siaran Pers' in data['news']:
        return True
    if '.id' in data['news']:
        return True
    
    return True

for n in tqdm(news):
    if 'disabled in your browser' in n['text'].lower():
        continue
        
    if len(n['text'].split()) > 1024:
        continue
        
    if n['language'] == 'ENGLISH':
        continue

    if reject(n):
        continue
    
    if n['url'] not in already:
        selected.append(n)
        already.add(n['url'])
        
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for n in data:
        if 'disabled in your browser' in n['text'].lower():
            continue
        
        if len(n['text'].split()) > 1024:
            continue
            
        if n['language'] == 'ENGLISH':
            continue
            
        if reject(n):
            continue
        
        if n['url'] not in already:
            selected.append(n)
            already.add(n['url'])
        
    
len(selected)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 974/974 [00:04<00:00, 206.72it/s]


81717

In [6]:
lens = [len(n['text'].split()) for n in selected]
max(lens)

1024

In [7]:
import numpy as np

np.argmax(lens)

57188

In [8]:
selected[-1]

{'title': 'Seni latte 3D',
 'url': 'https://www.hmetro.com.my/node/155492',
 'authors': [],
 'top-image': 'https://assets.hmetro.com.my/images/articles/senlatte_field_image_listing_featured.var_1469912795.jpg',
 'text': 'Seorang barista di Jepun mencipta hasil seni menarik dan comel hanya menggunakan buih susu dalam latte dihidangkannya.\n\nKazuki Yamamoto, 29, dari wilayah Okayama mencipta pelbagai haiwan kecil yang nampak realistik ketika bekerja di sebuah restoran, sebelum hasil seninya itu mula mendapat perhatian di seluruh dunia.\n\nKerja tangannya yang paling popular seekor kucing yang cuba masuk dari satu cawan ke cawan lain untuk menangkap ikan.\n\nDia yang kini pengurus sebuah kafe di Harajyuku juga membuat buih berbentuk arnab dan kucing yang lain, dianggap pelanggannya sebagai ‘terlalu comel sehingga tidak tergamak untuk meminumnya.’\n\n“Gambar kucing menangkap ikan menjadi popular selepas tersebar di Internet sehingga saya mendapat tawaran untuk menghadiri pelbagai acara da

In [9]:
with open('populate-news.json', 'w') as fopen:
    for s in selected:
        fopen.write(f'{json.dumps(s)}\n')